In [1]:
backup_dir = '/root/datos/maestria/netopaas/luca_explore/surgeries'
ikarus_dir = '/root/datos/maestria/netopaas/ikarus'
id_ = 'Chen_Zhang_2020_NSCLC'

In [2]:
# Parameters
id_ = "Maynard_Bivona_2020_NSCLC"


In [3]:
import gdown
import anndata as ad
import pandas as pd
import os

import scanpy as sc
from ikarus import classifier, utils, data

In [4]:
url = "https://raw.githubusercontent.com/BIMSBbioinfo/ikarus/master/tutorials/out/signatures.gmt"
signatures_path = f"{ikarus_dir}/signatures.gmt"
gdown.download(url, signatures_path, quiet=False) if not os.path.exists(signatures_path) else None
pd.read_csv(signatures_path, sep="\t", header=None)

model_path = f"{ikarus_dir}/core_model.joblib"

In [5]:
model = classifier.Ikarus(signatures_gmt=signatures_path, out_dir="out",
                          adapt_signatures=True # This is important, we are working with a reduced gen set, 
# so the model won't work if the intesrection betwen the siganture and the avialbe genes is too small..
# that is why the non-overlapping genes mus be removed from the signate. This flag does that automatically
                         )
model.load_core_model(model_path)

In [6]:
adata = sc.read_h5ad(f'{backup_dir}/filtered_{id_}.h5ad')
adata

AnnData object with n_obs × n_vars = 20592 × 55556
    obs: 'age', 'sex', 'well', 'plate', 'cell_id', 'sample', 'patient', 'race', 'smoking_status', 'condition', 'driver_gene', 'driver_mutation', 'secondary_mutation', 'Notes', 'stage_at_diagnosis', 'pathlogy_review', 'biopsy_date', 'sort_date', 'biopsy_type', 'tissue', 'origin', 'biopsy_time_status', 'early_treatment_status', 'best_response_status', 'biopsy_timing', 'analysis', 'treatment_history', 'treatment_history_detail', 'line_of_therapy', 'treatment_type', 'treatment', 'percent_PFS_ref_values', 'percent.PFS.reference.values', 'infections', 'early_bx_day', 'treatment_start_date', 'pfs_over_under', 'pfs_day', 'pfs_month', 'date_of_death', 'stageIII.IV_ca_dx_date', 'ca_dx_OS', 'multi_chain', 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3', 'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt', 'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr',

In [7]:
adata = data.preprocess_adata(adata)

In [8]:
adata.var['gene_symbol'] = adata.var.index
_ = model.predict(adata, "laughney",
                  # connectivities_path='scvi_conns.npz', # If we use the connectivites from scVI the number of predicted cells changes a lot
                  # save=True
                 )

/usr/local/lib/python3.11/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/usr/local/lib/python3.11/site-packages/scanpy/preprocessing/_highly_variable_genes.py:539: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['hvg'] = {'flavor': flavor}


2023-12-06 10:24:03.806816: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 10:24:03.806876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 10:24:03.808470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 10:24:03.817456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-12-06 10:24:05.072587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


converged at iteration step: 32 with 0.0009 < 0.001


In [9]:
preds = model.results["final_pred"].values

In [10]:
preds_df = pd.DataFrame({'final_pred':preds})
preds_df.to_csv(f'{ikarus_dir}/{id_}.csv')